In [27]:
import io
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

from bdd_100k_dataset_local import BDD100KDatasetLocal
from model import SmallUNet, SmallUNetSGM

import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics.segmentation as tm
import matplotlib.pyplot as plt

from torchmetrics.classification import BinaryF1Score, BinaryJaccardIndex

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
def load_data_local() -> pd.DataFrame:
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    train_dataset = BDD100KDatasetLocal(
        images_dir='./copied/Dataset/100k_images_train/bdd100k/images/100k/train',
        masks_dir='./copied/Dataset/bdd100k_lane_labels_trainval/bdd100k/labels/lane/masks/train',
        transform=transform
    )

    val_dataset = BDD100KDatasetLocal(
        images_dir='./copied/Dataset/100k_images_val/bdd100k/images/100k/val',
        masks_dir='./copied/Dataset/bdd100k_lane_labels_trainval/bdd100k/labels/lane/masks/val',
        transform=transform
    )

    test_dataset = BDD100KDatasetLocal(
        images_dir='./copied/Dataset/100k_images_test/bdd100k/images/100k/test',
        masks_dir='./copied/Dataset/bdd100k_lane_labels_trainval/bdd100k/labels/lane/masks/test',
        transform=transform
    )

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    return train_loader, val_loader, test_loader

In [29]:
train_loader, val_loader, test_loader = load_data_local()

model = SmallUNetSGM(in_channels=3, out_channels=1).cuda()
model.load_state_dict(torch.load('./best_smallunet_lane_detection_20.pth', map_location=device, weights_only = True)['model_state_dict'])

model.eval()

dice_lane = BinaryF1Score().to(device)
iou_lane = BinaryJaccardIndex().to(device)
dice_bg = BinaryF1Score().to(device)
iou_bg = BinaryJaccardIndex().to(device)

test_loader_tqdm = tqdm(test_loader, desc=f'Epoch {1}/{1} - Testing')
with torch.no_grad():
    for i, (images, masks) in enumerate(test_loader_tqdm):
        images = images.cuda()
        masks = masks.cuda()

        preds = model(images)
        targets=masks

        preds_bin = (preds > 0.5).int()

        # LANE: positive = 1
        dice_lane.update(preds_bin, masks)
        iou_lane.update(preds_bin, masks)

        # BACKGROUND: invert
        dice_bg[i].update(1 - preds_bin, 1 - masks)
        iou_bg[i].update(1 - preds_bin, 1 - masks)
        

        #Accuracy
        preds1_bin = (preds > 0.5).int()
        
        correct = (preds1_bin == targets).sum().item()
        total = targets.numel()
        # print("Acc2: ", correct/total)
        # correct = (preds2_bin == targets).sum().item()
        # print("Acc1: ", correct/total)

print("Accuracy: ", correct/total)
print("Dice Lane: ", dice_lane.compute().item())
print("Iou Lane: ", iou_lane.compute().item())
print("Dice Background: ", dice_bg.compute().item())
print("Iou Background: ", iou_bg.compute().item())

Epoch 1/1 - Testing: 100%|██████████| 1000/1000 [03:34<00:00,  4.66it/s]

Accuracy:  0.9891424179077148
Dice Lane:  0.6544666290283203
Iou Lane:  0.486399382352829
Dice Background:  0.9939548373222351
Iou Background:  0.9879823327064514


In [30]:
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm
from PIL import Image
from bdd_100k_dataset_local import BDD100KDatasetLocal
from torch.utils.data import DataLoader
from torchvision import transforms
import cv2

# --- Load Data ---
def load_data_local(batch_size=1):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),  # (C,H,W) normalized [0,1]
    ])
    test_dataset = BDD100KDatasetLocal(
        images_dir='./copied/Dataset/100k_images_test/bdd100k/images/100k/test',
        masks_dir='./copied/Dataset/bdd100k_lane_labels_trainval/bdd100k/labels/lane/masks/test',
        transform=transform
    )
    return DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# --- Load TFLite Model ---
interpreter = tf.lite.Interpreter(model_path="smallunet_model_quant.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_scale, input_zero_point = input_details[0]['quantization']
output_scale, output_zero_point = output_details[0]['quantization']

# --- Prepare ---
test_loader = load_data_local(batch_size=1)

dice_lane_vals = []
iou_lane_vals = []
dice_bg_vals = []
iou_bg_vals = []
lane_acc_vals = []

os.makedirs("mask_examples", exist_ok=True)
print("🔍 Evaluating TFLite model...\n")

# --- Evaluation Loop ---
for i, (x_batch, y_batch) in enumerate(tqdm(test_loader, desc="Testing", total=16000, leave=True, ncols=100)):
    # (N, C, H, W) --> (N, H, W, C)
    input_data = x_batch.permute(0, 2, 3, 1).numpy()  # shape (1, 256, 256, 3)
    # input_data = (input_data * 255.0).astype(np.float32)
    input_data = input_data.astype(np.float32)


    # Save a copy BEFORE quantization for visualization
    input_vis = np.clip(input_data[0], 0, 255).astype(np.uint8)  # (256,256,3)

    # Quantize for TFLite
    input_data = (input_data / input_scale + input_zero_point).clip(0, 255).astype(np.uint8)

    # Inference
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Dequantize output
    output_data = output_scale * (output_data.astype(np.float32) - output_zero_point)
    # print(f"[{i}] Output stats — min: {output_data.min():.6f}, max: {output_data.max():.6f}, mean: {output_data.mean():.6f}")

    # Threshold output
    y_pred_bin = (output_data > 0.5).astype(np.float32)
    y_pred_bin = y_pred_bin[0, 0, :, :]  # (256,256)

    # --- Metrics ---
    y_true = y_batch.numpy()[0, 0, :, :]
    y_true = (y_true > 0.1).astype(np.float32)

    intersection_lane = np.sum(y_true * y_pred_bin)
    union_lane = np.sum(y_true) + np.sum(y_pred_bin)
    dice_lane = (2. * intersection_lane + 1e-6) / (union_lane + 1e-6)
    iou_lane = (intersection_lane + 1e-6) / (union_lane - intersection_lane + 1e-6)

    y_true_bg = 1.0 - y_true
    y_pred_bg = 1.0 - y_pred_bin
    intersection_bg = np.sum(y_true_bg * y_pred_bg)
    union_bg = np.sum(y_true_bg) + np.sum(y_pred_bg)
    dice_bg = (2. * intersection_bg + 1e-6) / (union_bg + 1e-6)
    iou_bg = (intersection_bg + 1e-6) / (union_bg - intersection_bg + 1e-6)

    correct_pixels = np.sum(y_true == y_pred_bin)
    total_pixels = np.prod(y_true.shape)
    lane_acc = correct_pixels / total_pixels

    dice_lane_vals.append(dice_lane)
    iou_lane_vals.append(iou_lane)
    dice_bg_vals.append(dice_bg)
    iou_bg_vals.append(iou_bg)
    lane_acc_vals.append(lane_acc)

# --- Results ---
print("\n📊 Final TFLite Evaluation:")
print(f"🚦 Lane Accuracy: {np.mean(lane_acc_vals):.4f}")
print(f"🎯 Dice Lane: {np.mean(dice_lane_vals):.4f} | IoU Lane: {np.mean(iou_lane_vals):.4f}")
print(f"🧱 Dice Bg:   {np.mean(dice_bg_vals):.4f} | IoU Bg:   {np.mean(iou_bg_vals):.4f}")

🔍 Evaluating TFLite model...



Testing:  91%|█████████████████████████████████████████▊    | 14542/16000 [1:06:05<06:54,  3.51it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Testing: 100%|██████████████████████████████████████████████| 16000/16000 [1:12:42<00:00,  3.67it/s]


📊 Final TFLite Evaluation:
🚦 Lane Accuracy: 0.9882
🎯 Dice Lane: 0.6107 | IoU Lane: 0.4659
🧱 Dice Bg:   0.9940 | IoU Bg:   0.9880
